In [ ]:
# !pip install ipynb
# from ipynb.fs.full.useful_functions import *

from ipynb.fs.full.functions_for_QAOAAnsatz import *

In [ ]:
max_p = 3
random_attempts = 10

In [ ]:
# current_datetime = datetime.now().strftime("@%Y-%m-%d@%Hh%Mm%Ss")
current_datetime = datetime.now().strftime("%d-%m")

# This file will contain the final histograms.
header = f"{current_datetime}_dim{n}_mail{instance}_{init_name}_maxp{max_p}_{random_attempts}ra_k{k}_BOUNDS[0,pi]x[0,2pi]"


# This file will contain important data such as energies, betas, gammas.
from ast import literal_eval
DATA_FILENAME = header + '_data.txt'
FILENAME = header + ".csv"

print(FILENAME, "\n", DATA_FILENAME)

In [ ]:
# p=3
# ansatz = QAOAAnsatz(qc_ham, mixer_operator=qc_mixing, initial_state=q_initial, reps=p, name='my_QAOA_circuit')

# # Generate a pass manager without providing a backend
# pm = generate_preset_pass_manager(optimization_level=3)
# ansatz_isa = pm.run(ansatz)
# hamiltonian_isa = hamiltonian.apply_layout(ansatz_isa.layout)

# estimator = StatevectorEstimator()
# sampler = StatevectorSampler()


# *******************************  MINIMIZE  **********************************
# # Do the minimization.

# pars_0 =  [np.pi * random.random(), np.pi * random.random(), 2 * np.pi * random.random(), 2 * np.pi * random.random()] 
    
# print('\nInitial parameters (for the minimization): pars_0 = ', pars_0)

# bnds = [(0, np.pi) for _ in range(p)] + [(0, 2*np.pi) for _ in range(p)]
# print("\nbnds", bnds)
# res = minimize(cost_func, pars_0, bounds=bnds,
#                args=(ansatz_isa, hamiltonian_isa, estimator), 
#                method="Nelder-Mead", options={"maxiter": 1000}, tol=1e-4)

# print("\nres", res)

# # Select the optimal parameters [beta,gamma] found.
# print("\nres.x", res.x)
# betas = list(res.x[:p])
# gammas = list(res.x[p:])
# print(f"Final parameters (after minimization): betas, gammas = {betas}, {gammas}")


# ****************************  RUN THE CIRCUIT  ******************************
# # Assign to the ansatz the 2p parameters found, then run the circuit.
# pars =[-1.54615350851878, 0.909773247370973, 1.784442449592837]+[-35.582930846019636, 51.45376690589117, 71.26888048656258]
# print("\npars", pars)
# qc = ansatz.assign_parameters(pars)
# qc.measure_all()
# qc.decompose(reps=2).draw('mpl')

# qc_isa = pm.run(qc)
# result = sampler.run([qc_isa], shots=1024).result()
# samp_dist = result[0].data.meas.get_counts()


# # ****************************  POST PROCESS  **********************************
# # Create a dataframe out of the sampling's results.
# df = pd.DataFrame(samp_dist.items()).rename(columns={0: 'states', 1: f'counts_p{p+1}'})
# df['states'] = df['states'].apply(lambda x: x[NUM_ANC:]) # remove ancillary bits
# df = df.groupby(['states']).sum()

# # Create a dictionary with states and occurrences.
# d = df[f'counts_p{p+1}'].to_dict()
# lists = sorted(d.items(), key=lambda item: item[1], reverse=True)

# # Invert bit order ("01101 -> 10110")      
# d = invert_counts(d)


# # **************************** PLOT THE i-TH DATAFRAME ****************************
# plt.figure(figsize=(15,5))
# plt.rcParams['font.size'] = 13
# plt.title(f"Ansatz with p={p}")
# df = pd.DataFrame(d.items())
# df = df.rename(columns={0: 'states', 1: f'counts_p{p}'})    
# df = df.sort_values(f'counts_p{p}', ascending=False) 

# ax = sns.barplot(x='states', y=f'counts_p{p}', data=df)

# # Make labels with percentages.
# labels = df[f'counts_p{p}'].apply(lambda x: (x/df[f'counts_p{p}'].sum())*100).round(1).astype('str') + '%'
# for container in ax.containers:
#     ax.bar_label(container, labels=labels, fontsize=11)
    
# # Highlight with red the exact covers
# highlight_correct_ticks(ax, df, EXACT_COVERS)
# plt.show()

In [ ]:
# ansatz = QAOAAnsatz(qc_ham, mixer_operator=qc_mixing, initial_state=q_initial, reps=p, name='my_QAOA_circuit')
# ansatz.parameter_bounds=[[0, 2*np.pi], None]
# print(ansatz.parameter_bounds)

In [ ]:
with open(DATA_FILENAME, 'a') as DATA_FILE:
    DATA_FILE.write(f"The initial state is an equal superposition of: {init_state} \n")
    
    DATA_FILE.write(f"max_p={max_p}\n")

    Energies = []
    
    TOTAL_start_time = time.time()
    DATA_FILE.write(f"Random attempts for each layer: {random_attempts}\n")
    
    for p in range(1, max_p+1):
        
        print(f"\n **** p = {p} **** ")
        pth_start_time = time.time()
        E_best = 100
        counter = 0
    
        
        Energies_of_layer_p = []
        
        for i in range(random_attempts):
            print(f"-------------{i}/{random_attempts}-----------------")
            ansatz = QAOAAnsatz(qc_ham, mixer_operator=qc_mixing, initial_state=q_initial, reps=p, name='my_QAOA_circuit')
            ###############
            ansatz.parameter_bounds=[(0, np.pi)]*p + [(0, 2*np.pi)]*p
            
            # Generate a pass manager without providing a backend
            pm = generate_preset_pass_manager(optimization_level=3)
            ansatz_isa = pm.run(ansatz)
            hamiltonian_isa = hamiltonian.apply_layout(ansatz_isa.layout)
        
            estimator = StatevectorEstimator()
            sampler = StatevectorSampler()
    
            
            # *******************************  MINIMIZE  **********************************
            # Do the minimization.
            ###############
            if p == 1:
                pars_0 = [np.pi * random.random(), 2 * np.pi * random.random()]
                bnds = [(0, np.pi), (0, 2*np.pi)]
            else:
                pars_0 =  pth_betas + [np.pi * random.random()] + pth_gammas + [2 * np.pi * random.random()] 
                bnds = [(0, np.pi)]*p + [(0, 2*np.pi)]*p
    
                
            print('Initial parameters (for the minimization): pars_0 = ', pars_0)
            ###############
            # bnds = [(0, np.pi) for _ in range(p)] + [(0, 2*np.pi) for _ in range(p)]
            # bnds = [(0, 2*np.pi)] * p + list(np.empty(p, dtype=object))
            print(bnds)
            res = minimize(cost_func, pars_0, bounds=bnds,
                           args=(ansatz_isa, hamiltonian_isa, estimator), 
                           method="Nelder-Mead", options={"maxiter": 1000}, tol=1e-4)
        
            
            # Select the optimal parameters [beta,gamma] found.
            betas = list(res.x[:p])
            gammas = list(res.x[p:])
            print(f"Final parameters (after minimization): betas, gammas = {betas}, {gammas}")
    
            
            # ****************************  RUN THE CIRCUIT  ******************************
            # Assign to the ansatz the 2p parameters found, then run the circuit.
            pars = betas + gammas
            qc = ansatz.assign_parameters(pars)
            qc.measure_all()
            qc_isa = pm.run(qc)
            result = sampler.run([qc_isa], shots=1024).result()
            samp_dist = result[0].data.meas.get_counts()
    
            
            # ****************************  POST PROCESS  **********************************
            # Create a dataframe out of the sampling's results.
            df = pd.DataFrame(samp_dist.items()).rename(columns={0: 'states', 1: f'counts_p{p+1}'})
            df['states'] = df['states'].apply(lambda x: x[NUM_ANC:]) # remove ancillary bits
            df = df.groupby(['states']).sum()
            
            # Create a dictionary with states and occurrences.
            d = df[f'counts_p{p+1}'].to_dict()
            lists = sorted(d.items(), key=lambda item: item[1], reverse=True)
        
            # Invert bit order ("01101 -> 10110")      
            d = invert_counts(d)
    
        
            # **************************** PLOT THE i-TH DATAFRAME ****************************
            plt.figure(figsize=(15,5))
            plt.rcParams['font.size'] = 13
            plt.title(f"Ansatz with p={p} iteration {i}/{random_attempts}")
            df = pd.DataFrame(d.items())
            df = df.rename(columns={0: 'states', 1: f'counts_p{p}'})    
            df = df.sort_values(f'counts_p{p}', ascending=False) 
            
            ax = sns.barplot(x='states', y=f'counts_p{p}', data=df)
            
            # Make labels with percentages.
            labels = df[f'counts_p{p}'].apply(lambda x: (x/df[f'counts_p{p}'].sum())*100).round(1).astype('str') + '%'
            for container in ax.containers:
                ax.bar_label(container, labels=labels, fontsize=11)
                
            # Highlight with red the exact covers
            highlight_correct_ticks(ax, df["states"], EXACT_COVERS)
            plt.show()
            
            # ******************* UPDATE PARAMETERS IF ENERGY HAS DECREASED *******************
            E_min = res.fun
            print(f"E_min - A - B = {E_min-A-B}")
            Energies_of_layer_p.append(E_min-A-B)
            
            if E_min < E_best:
                print("...UPDATING...")
                E_best = E_min
                d_best = d
                betas_best = betas
                gammas_best = gammas
    
            if df['states'].iloc[0] == MEC:
                print("### Most frequent state is MEC ###")
                counter += 1
            # if compute_energy_Lucas(df['states'].iloc[0], U, subsets_dict) == 0:
            #     print("### Most frequent state is an EC ###")
            #     counter += 1
            #     if df[f'counts_p{p}'].iloc[0] == 1024:
            #         print("-- BREAKING CYCLE - EXACT COVER FOUND --")
            #         break
                    
            # *********************************************************************************
    
        Energies.append(Energies_of_layer_p)
        DATA_FILE.write(f"\n----- p = {p} layer -----\nMinimum energy found in each attempt: ")
        DATA_FILE.write(str(Energies_of_layer_p))
        
        ################### PLOT THE p-TH DATAFRAME ###################
        # Plot the dataframe.
        plt.figure(figsize=(15,5))
        plt.rcParams['font.size'] = 13
        plt.title(f"Best parameters' ansatz with p={p}")
        df = pd.DataFrame(d_best.items())
        df = df.fillna(0)
        df = df.rename(columns={0: 'states', 1: f'counts_p{p}'})    
        df = df.sort_values(f'counts_p{p}', ascending=False)
        ax = sns.barplot(x='states', y=f'counts_p{p}', data=df, color='red')
    
        # Make labels with percentages.
        labels = df[f'counts_p{p}'].apply(lambda x: (x/df[f'counts_p{p}'].sum())*100).round(1).astype('str') + '%'
        for container in ax.containers:
            ax.bar_label(container, labels=labels, fontsize=11)
        
        highlight_correct_ticks(ax, df["states"], EXACT_COVERS)
        plt.show()
    
        # ################### SAVE TO CSV AND FIX BETA,GAMMA ###################
        # Save to csv.
        df.to_csv(FILENAME + f"_p{p}.csv", index=False)

        print(f"E_best - A - B = {E_best - A - B}")
        DATA_FILE.write(f"\nE_best = {E_best - A - B}")
        
        # Best parameters'vector of length 2p found for the p-th cycle
        pth_betas = betas_best
        pth_gammas = gammas_best
        print(f'\nBEST PARAMETERS OF p={p} LAYER: betas, gammas = ', pth_betas, pth_gammas)
        DATA_FILE.write(f'\nE_best\'s parameters: betas = {pth_betas}, gammas = {pth_gammas}\n')
        
        print("Execution time of p={0} layer: {1:.1f} min".format(p, (time.time() - pth_start_time)/60))
        DATA_FILE.write("Execution time of p={0} layer: {1:.1f} min\n".format(p, (time.time() - pth_start_time)/60))
        print(f"Most frequent state was MEC / random attempts = {counter} / {random_attempts}") 
        DATA_FILE.write(f"Most frequent state was MEC / random attempts = {counter} / {random_attempts} = {round((counter/random_attempts)*100, 1)}%\n")
    
        # Merge dataframes.
        if p == 1:
            df_final = df
        else:
            df_final = pd.merge(df_final, df, on="states", how="outer")    
    
        
    # ************************ SEE FINAL RESULT ************************
    # Plot all the dataframes in one picture.
    plt.figure()
    plt.rcParams['font.size'] = 13
    df_final = df_final.fillna(0)
    df_final.plot(x='states', kind="bar", figsize=(20,5), stacked=False)
    
    # Make labels with percentages.
    ax1 = plt.gca()
    ax1.set_ylabel("counts")
    for i,container in enumerate(ax1.containers):
        p = i+1
        labels = df_final[f'counts_p{p}'].apply(lambda x: (x/df_final[f'counts_p{p}'].sum())*100).round(1).astype('str') + '%'
        ax1.bar_label(container, labels=labels)
    highlight_correct_ticks(plt.gca(), df_final["states"], EXACT_COVERS)
    
    
    ######################################################################
    print(f"\nTOTAL ELAPSED TIME: {(time.time() - TOTAL_start_time)/60} minutes.")
    DATA_FILE.write(f"\nTOTAL ELAPSED TIME: {(time.time() - TOTAL_start_time)/60} minutes.\n")

In [ ]:
print(p)
ansatz.parameter_bounds

### energia minima al variare di p

In [ ]:
"""
Questo è il grafico di quello che su Lee (2021) viene chiamato "mean approximation ratio".
Non credo che però in questo problema abbia senso. L'energia potrebbe diminuire anche nel 
caso in cui finisse in un minimo locale, se il minimo locale ha energia abbastanza bassa.
"""

E_mean = []
E_min = []
E_std = []
E_max = []

for Energies_of_layer_p in Energies:
    e = Energies_of_layer_p
    print(e)
    E_mean.append(np.mean(e))
    E_min.append(np.min(e))
    E_std.append(np.std(e))
    E_max.append(np.max(e))

plt.figure()
plt.grid()
plt.xlabel("p")
plt.ylabel(f"Mean over {random_attempts} mean energies")
p = np.array([str(i+1) for i in np.arange(len(Energies))])
plt.errorbar(x=p, y=E_mean, yerr=E_std, 
             elinewidth=3, capsize=5, barsabove=True, color = 'black', fmt ='o')
plt.plot(p, E_min, color = 'red', linestyle='--', marker="o", label='Minimum values')
plt.plot(p, E_max, color = 'blue', linestyle='--', marker="o", label='Maximum values')

plt.ylim(-1,-0.95)
plt.legend(loc="best")
print("\n E_min = ", E_min)

## Lettura da file

In [ ]:
# ############# DIM 6 #############
all_instances = {
1: [{4, 6, 7, 9, 10, 11}, {1, 2, 5, 6, 11, 12}, {8, 1, 12}, {2, 3, 5}, {1, 3, 4, 5, 9, 12}, {2, 6, 7, 9, 12}] ,
2: [{2, 11, 12, 6}, {2, 4, 6, 8, 9, 11}, {1, 3, 5, 7, 10, 12}, {2, 7}, {2, 3, 4, 5, 8, 12}, {1, 2, 8, 9, 12}] ,
3: [{8, 10, 3}, {2, 4, 5, 6, 9, 11, 12}, {1, 3, 7, 8, 10}, {3, 4, 6, 8, 11}, {2, 3, 4, 6, 7, 9, 12}, {1, 7}] ,
4: [{1, 4, 5, 6, 8, 11, 12}, {3, 6, 8, 9, 10}, {1, 2, 11, 5}, {2,3, 7, 9,10}, {8, 3, 12, 6}, {4, 7, 12}] ,
5: [{11, 7}, {1, 2, 4, 5, 9, 11}, {1, 3, 4, 6, 8}, {2, 5}, {3, 6, 7, 8,10, 12}, {9, 10, 12}] ,
6: [{2, 3, 4, 5, 7, 8,9, 10, 11, 12}, {5, 9, 10, 11}, {3, 6, 7, 8}, {1, 2, 4, 12}, {1, 6}, {2, 3, 4, 7, 8, 12}] ,
7: [{1, 2, 5, 6, 7, 9, 12}, {10, 3, 4}, {2, 5, 7, 8, 9, 10, 11}, {8, 11}, {1, 7, 8, 11, 12}, {11, 9, 3, 7}] ,
8: [{1, 2, 4, 5, 6, 8, 9, 10}, {1, 4, 6, 7}, { 5, 8, 9, 11, 12}, {4, 7, 8, 9, 10, 11, 12}, {2, 3, 4, 5, 6, 7, 11, 12}, {10, 2, 3}] ,
9: [{1, 4, 7, 9, 10, 11, 12}, {2, 3, 5, 6, 8}, {1, 2, 4, 9, 12}, {2,3, 6, 7, 8, 9}, {1, 4, 6, 7, 8, 10, 11}, {8, 11, 3, 6}] ,
10: [{1, 11, 12, 6}, {2, 5, 8, 10, 11, 12}, {2, 6, 7, 8, 9, 10, 11}, {2, 3,4, 5, 7, 8, 9, 10}, {1, 2, 4, 5, 6, 7, 8, 11}, {1, 5, 6, 9, 10, 11, 12}]} 

# ########### DIM 8 ###########
# all_instances = { 
# 1: [{1, 2, 3, 4, 7, 8, 9, 12, 13, 14}, {1, 4, 5, 8, 9, 12, 13, 14, 16}, {1, 6, 9, 10, 11, 13, 15, 16}, {5, 6, 10, 11, 15, 16}, {1, 6, 7, 8, 9, 10, 14, 15}, {8, 13, 4, 12}, {2, 3, 4, 5, 7, 8, 11, 12, 13}, {4, 5, 6, 7, 10, 12, 14, 16}],
# 2: [{1, 2, 3, 5,7,12, 15}, {6, 7, 8, 9}, {4, 6, 8, 9, 10, 11, 13, 14, 16}, {3, 4, 5, 6, 7, 9, 10, 11, 12, 15}, {4, 12, 14}, {2, 3, 7, 8, 10, 15}, {1, 4, 6, 8, 9, 10, 12, 14, 15}, {1, 2, 3, 5, 10, 11,13, 15, 16}],
# 3: [{4, 7, 14, 15,16}, { 4, 5, 8, 9, 10, 13, 14, 16}, {1, 3, 4, 6, 8, 9, 10, 12, 13, 14}, {1, 3, 5, 6, 10}, {2, 4, 6, 9, 10, 11, 15}, {1, 2, 4, 5, 9, 13, 14, 15}, {1, 2, 3, 6, 7, 11, 12, 15}, {2, 8, 9, 11, 12, 13}], 
# 4: [{2, 4, 5, 6, 9}, {13, 14, 15, 16}, {1, 4, 5, 6}, { 7, 8, 10, 12}, {1, 3, 7, 8, 10, 11, 12, 13, 14, 15, 16}, {2, 3, 9, 11}, {3, 4, 5, 7, 8, 10, 12}, {3, 7, 8, 9, 10, 11, 14}],
# 5: [{5, 8, 10, 11, 12, 13, 16}, {1, 2, 3, 4, 6, 7, 9, 14, 15}, {1, 2, 5, 7, 8, 9, 10, 11, 12, 13, 15, 16}, {3, 4, 9, 11, 13, 14, 16}, {2, 3, 6, 10, 13, 16}, {1, 3, 8, 10, 11, 13,14, 15}, {1, 6, 8, 10, 11, 13, 14, 16}, {3, 4, 7, 10, 11, 13, 14, 15, 16}],
# 6: [{1, 2, 5, 6, 7, 10, 12, 15}, {5, 6, 9, 10}, {2, 7, 11, 12}, {3, 6, 7, 8, 9, 12, 15, 16}, {1, 3, 4, 8, 13, 14, 15, 16}, {2, 3, 6, 7, 8, 10, 15, 16}, {1, 2, 3, 4, 7, 8, 12, 13, 14}, {2, 3, 8, 11, 12, 14}],
# 7: [{2, 3, 5, 6, 7, 8, 10, 11, 12, 13, 15}, {2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 15, 16}, {10, 11, 15, 7}, {2, 3, 9, 10, 11, 14, 16}, {1, 2, 7, 10, 11, 12, 13, 14, 15}, {1, 4, 9, 14, 16}, {3, 4, 5, 9, 11, 12, 14, 16}, {2, 7, 9, 10, 11, 12, 13, 14, 15, 16}],
# 8: [{1, 4, 8, 9, 10, 13, 14, 15, 16}, {1, 2, 4, 5, 6, 7, 9, 12, 13}, {2, 3, 5, 6, 7, 11, 12}, { 5, 11, 12}, {1, 2, 3, 7, 8, 10, 11, 13}, {1, 2, 4, 7, 8, 13, 14, 15, 16}, {2, 3, 6, 7}, {1, 3, 4, 6, 9, 11, 12, 14, 16}],
# 9: [{2, 4, 14,15}, {1, 3, 6, 8}, {5, 7, 10, 13, 16}, {1, 2, 4, 6, 7, 10, 11, 13, 15, 16}, {4, 6, 7, 11, 12, 13, 15}, {3, 5, 8, 9, 12, 14}, {9, 11, 12}, {2, 3, 4, 5, 7, 8, 9, 10, 12, 14, 15, 16}],
# 10: [{2, 3, 5, 6, 7, 8, 9, 11, 13, 16}, {1, 4, 5, 7, 9, 12, 14}, {6, 7, 11, 13, 14, 15, 16}, {6, 7, 8, 9, 11, 15, 16}, {2, 5, 10, 12, 14}, {1, 5, 6, 7, 11, 12, 14}, {2, 3, 4, 5, 7, 11, 12, 13, 14, 15, 16}, {1, 3, 4, 13}]
# }

instance = 3

subsets_dict = dict([(k,v) for k,v in enumerate(all_instances[instance])])
n = len(subsets_dict)

# Scelta l'istanza si può calcolare U.
U = find_U_from_subsets(subsets_dict)


print("subsets:\n", subsets_dict)
print("n:\n", n)
print("U:\n", U)




###############################

EXACT_COVERS = []
for nuple in bit_gen(n):
    
    state = "".join([str(bit) for bit in nuple]) # strings
    E = compute_energy_Lucas(state, U, subsets_dict)
    if E == 0: EXACT_COVERS.append(state)

MECs = [state for i,state in enumerate(EXACT_COVERS) if state.count("1") == min([x.count("1") for x in EXACT_COVERS])]
print("MEC", MECs)

ECs = [elem for elem in EXACT_COVERS if elem!=MEC]
print("EC", ECs)


In [ ]:
header = "11-09_dim6_mail3_all0_maxp3_10ra_k0.067_BOUNDS[0,pi]x[0,2pi]"
max_p = 3

In [ ]:
most_frequent_states = []
labels = []

for p in range(1, max_p+1):

    # Read from csv.
    file_to_read = header + f"_p{p}.csv"
    
    df = pd.read_csv(file_to_read, dtype = {'states':"str", f'counts_p{p}':"Int64"})
    most_frequent_states.append([df["states"].iloc[0], df[f'counts_p{p}'].iloc[0]])
    
    # Plot the dataframe.
    plt.figure(figsize=(15,5))
    df = df.sort_values(f'counts_p{p}', ascending=False)
    ax = sns.barplot(x='states', y=f'counts_p{p}', data=df)
    highlight_correct_ticks(ax, df["states"], EXACT_COVERS)
    
    # Make labels with percentages.
    labels.append(df[f'counts_p{p}'].apply(lambda x: (x/df[f'counts_p{p}'].sum())*100).round(1).astype('str') + '%')
    for container in ax.containers:
        ax.bar_label(container, labels=labels[p-1])
    plt.show()
    
    # Merge dataframes.
    if p == 1:
        df_final = df
    else:
        df_final = pd.merge(df_final, df, on="states", how="outer")    


# Plot all the dataframes in one picture.
plt.figure(figsize=(15,5))
df_final.plot(x='states', kind="bar", figsize=(15,10), fontsize=8, stacked=False)
df_final = df_final.fillna(0)

# Make labels with percentages.
ax1 = plt.gca()
ax1.set_ylabel("counts")
for i,container in enumerate(ax1.containers):
    p = i+1
    labels = df_final[f'counts_p{p}'].apply(lambda x: (x/df_final[f'counts_p{p}'].sum())*100).round(1).astype('str') + '%'
    ax1.bar_label(container, labels=labels)
highlight_correct_ticks(plt.gca(), df_final["states"], EXACT_COVERS)

In [ ]:
# Find the exact cover 
mask = df_final['states'].isin(ECs)
print(df_final.head())
print(mask)

df_exact_covers = df_final[mask]
print(df_exact_covers.head())
exact_covers = df_exact_covers["states"].values
print(exact_covers)
solution =  df_exact_covers["states"].iloc[0]
print(solution)
counts = df_exact_covers.iloc[0, 1:].values
layers = [str(i+1) for i in np.arange(max_p)]

# Plot the figure.
plt.figure()
total = 1024
plt.plot(layers, counts*100/total, linestyle='--', color='k', marker='o', label=f"{solution}")
plt.ylabel("Occurrences [%]")
plt.xlabel("p (number of layers)")
plt.legend()

plt.show()

In [ ]:
probs = []
mask = [] # True if the most frequent state is the correct state, False otherwise

total = 1024

for tup in most_frequent_states:
    state = tup[0]
    count = tup[1]
    if(state in ECs):
        mask.append(True)
    else:
        mask.append(False)
        
    probs.append(100 * count / total)

print(probs, mask)
mask = np.array(mask)
probs = np.array(probs)

# ********************* plot the figure *******************************
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import MaxNLocator

fig, ax = plt.subplots()
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

p = np.arange(1,max_p+1)
plt.errorbar(p[mask], probs[mask], fmt='o', color='g', label='Was the solution')
plt.errorbar(p[~mask], probs[~mask], fmt='o',  color='r', label='Was not the solution')
plt.plot(p, probs, linestyle='--', color='k', alpha = 0.5)

plt.ylabel("Occurrences [%]")
plt.xlabel("p")
plt.legend()
plt.title("Most frequent states")
plt.show()